# Simplest Example

In [46]:
import numpy as np

A = np.mat([
    [0,1,0,0],
    [0,0,1,1],
    [0,1,0,0],
    [1,0,1,0]
])

X= np.mat([
    [i, -i]
    for i in range(A.shape[0])
], dtype = float)

I = np.mat(np.eye(A.shape[0]))

A_hat = A + I

D = np.array(A.sum(axis=0))[0]
D = np.matrix(np.diag(D))
D_inverse = D**-1

D_hat = np.array(A_hat.sum(axis=0))[0]
D_hat = np.matrix(np.diag(D_hat))
D_hat_inverse = D_hat**-1

W = np.matrix([
    [1, -1],
    [-1, 1]
])

print("A")
display(A)

print("X")
display(X)

print("D")
display(D)

print("D_inverse")
display(D_inverse)

print("A_hat")
display(A_hat)

print("D_hat")
display(D_hat)

print("D_hat_inverse")
display(D_hat_inverse)


A


matrix([[0, 1, 0, 0],
        [0, 0, 1, 1],
        [0, 1, 0, 0],
        [1, 0, 1, 0]])

X


matrix([[ 0.,  0.],
        [ 1., -1.],
        [ 2., -2.],
        [ 3., -3.]])

D


...

matrix([[1., 1., 0., 0.],
        [0., 1., 1., 1.],
        [0., 1., 1., 0.],
        [1., 0., 1., 1.]])

D_hat


matrix([[2., 0., 0., 0.],
        [0., 3., 0., 0.],
        [0., 0., 3., 0.],
        [0., 0., 0., 2.]])

D_hat_inverse


matrix([[0.5       , 0.        , 0.        , 0.        ],
        [0.        , 0.33333333, 0.        , 0.        ],
        [0.        , 0.        , 0.33333333, 0.        ],
        [0.        , 0.        , 0.        , 0.5       ]])

In [47]:
def relu_arr(x):
    return x * (x > 0)

In [48]:
def relu_mat(x):
    return np.maximum(x, 0)

In [50]:
relu_mat(D_hat_inverse * A_hat * X * W)

matrix([[1., 0.],
        [4., 0.],
        [2., 0.],
        [5., 0.]])

# Zachary karate Club

In [51]:
from networkx import karate_club_graph, to_numpy_matrix

zkc = karate_club_graph()
order = sorted(list(zkc.nodes()))
A = to_numpy_matrix(zkc, nodelist=order)
I = np.eye(zkc.number_of_nodes())

A_hat = A + I
D_hat = np.array(np.sum(A_hat, axis=0))[0]
D_hat = np.matrix(np.diag(D_hat))

In [52]:
W_1 = np.random.normal(
    loc=0, scale=1, size=(zkc.number_of_nodes(), 4))

W_2 = np.random.normal(
    loc=0, size=(W_1.shape[1], 2))

In [54]:
def gcn_layer(A_hat, D_hat, X, W):
    return relu_mat(D_hat**-1 * A_hat * X * W)

H_1 = gcn_layer(A_hat, D_hat, I, W_1)
H_2 = gcn_layer(A_hat, D_hat, H_1, W_2)

output = H_2

In [62]:
feature_representations = {
    node: np.array(output)[node] 
    for node in zkc.nodes()}

feature_representations

for node in zkc.nodes():
    print(np.array(output)[node])
    print(zkc.node[node]['club'])

[0.16777281 0.18458304]
Mr. Hi
[0.27213135 0.3556863 ]
Mr. Hi
[0.13470072 0.37249101]
Mr. Hi
[0.2151584  0.27618064]
Mr. Hi
[0.01652582 0.        ]
Mr. Hi
[0.02220998 0.        ]
Mr. Hi
[0.03138302 0.        ]
Mr. Hi
[0.23127391 0.30925774]
Mr. Hi
[0.10236214 0.03157763]
Mr. Hi
[0.12869859 0.2448664 ]
Officer
[0.00620875 0.        ]
Mr. Hi
[0.07880765 0.01439395]
Mr. Hi
[0.17478569 0.18081699]
Mr. Hi
[0.14813422 0.1076453 ]
Mr. Hi
[0.06118947 0.        ]
Officer
[0. 0.]
Officer
[0.0287383 0.       ]
Mr. Hi
[0.21794101 0.27811617]
Mr. Hi
[0. 0.]
Officer
[0.12230777 0.02078251]
Mr. Hi
[0. 0.]
Officer
[0.24110205 0.31917346]
Mr. Hi
[0.         0.24173301]
Officer
[0.11560116 0.1097871 ]
Officer
[0.05878408 0.20501358]
Officer
[0.12318424 0.        ]
Officer
[0.18245963 0.4059117 ]
Officer
[0.12503199 0.16904053]
Officer
[0.1046182 0.       ]
Officer
[0.16040339 0.20601713]
Officer
[0.08058651 0.        ]
Officer
[0.04170478 0.        ]
Officer
[0.08097795 0.        ]
Officer
[0.09290443 0